# Notebook Context

For context this is 3 of 5 Jupyter Notebooks associated with the used car project

This Notebook covers the GeoCoding aspect of the project which in the end, was found to not improve model performance at all. How disappointing.

# GeoCoding to obtain latitude and longitude from car listing information

In [25]:
# Import packages

import pandas as pd
import glob
import numpy as np
import regex
#import reverse_geocoder as rg
from tqdm import tqdm
import urllib.parse
import datetime 
import time
from bs4 import BeautifulSoup
import warnings
import pprint
import googlemaps
from geopy.geocoders import Nominatim
from datetime import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Import cleaned cars data

abspath = r'/Users/robertdavies/Desktop/DSI/GA_P2/LargeDatasets/00_all_cars.csv'
df = pd.read_csv(abspath)
df.sample(5)

,name,name_subtitle,year,price,body,mileage,BHP,doors,transmission,make,...,seller1,used,e_engine_kWh,log_price,log_mileage,orig_name,id,year_reg,link,href0
368443,mercedes-benz a class,1.3 A180 AMG Line (Executive) 7G-DCT (s/s) 4dr,2021,31699.0,Saloon,1083.0,134.122288,4dr,Automatic,Mercedes-Benz,...,4.9,1,NaN,10.364040,6.987490,Mercedes-Benz A Class 1.3 A180 AMG Line (Execu...,202108316810864,NaN,https://www.autotrader.co.uk/car-details/20210...,/dealers/county-down/newry/shelbourne-motors-a...
25853,volkswagen golf,2.0 TDI GT 5dr,2010,3699.0,Hatchback,128445.0,138.000000,5dr,Manual,Volkswagen,...,NaN,1,NaN,8.215818,11.763256,Volkswagen Golf,202109237731280,2010 (60 reg),https://www.autotrader.co.uk/car-details/20210...,NaN
161098,kia picanto,1.0 1 5dr,2017,7330.0,Hatchback,26247.0,66.074951,5dr,Manual,KIA,...,3.4,1,NaN,8.899731,10.175307,KIA Picanto,202112180626837,2017 (67 reg),https://www.autotrader.co.uk/car-details/20211...,/dealers/peterborough/fengate/car-store-peterb...
97332,vauxhall mokka,1.6 EXCLUSIV S/S 5d 113 BHP,2015,8450.0,SUV,60000.0,113.000000,5dr,Manual,Vauxhall,...,5.0,1,NaN,9.041922,11.002100,Vauxhall Mokka,202112160586613,2015 (15 reg),https://www.autotrader.co.uk/car-details/20211...,/dealers/wiltshire/salisbury/landford-car-cent...
54756,bmw z4,2.0 20i M Sport sDrive (s/s) 2dr,2013,17100.0,Convertible,27600.0,184.000000,2dr,Automatic,BMW,...,NaN,1,NaN,9.746834,10.225571,BMW Z4,202201181481997,2013 (62 reg),https://www.autotrader.co.uk/car-details/20220...,NaN


In [3]:
# Define function which uses nominamtim to return lat/lon for each car area

# This function takes a low (l) and high (h) index as parameters for which to return latitude and longitude
# It takes a VERY long time, at approximately 2s per car. I would recommend running it in swaths.
# The function will output a csv containing car.name, car.id, car.dealer_city, car.dealet_lat and car.dealer_lon
# which can be merged with the cleaned car data. This process is done in Notebook 04.
def get_latlon_to_csv_used(l, h):
    lat_series = []
    lon_series = []
    # dealer city holder
    dc = []
    # car ID holder
    cid = []
    name = []
    # subset the car dataframe based on the index of rows that you want to obtain lat lon for
    ldf = df.loc[l:h,['name','id','dealer_city']]

    for index, car in ldf.iterrows():
        try:
            address = car[-1] + ' UK'
            # Indert the dealer_city and 'UK' into the nominatim search
            url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
            response = requests.get(url).json()
            # Extract the relevant lat/lon data from the json response.
            lat_series.append(' '+str(response[0]["lat"]))
            lon_series.append(' '+str(response[0]["lon"]))
            # Append corresponding car details for merge later
            cid.append(car[-2])
            dc.append(car[-1])
            name.append(car[-3])
        except:
            pass

    # throw complete entries into a dataframe and output to csv.     
    odf = pd.DataFrame({'name': name,
                       'id': cid,
                       'dealer_city':dc,
                       'dealer_lat':lat_series,
                        'dealer_lon':lon_series})    
    now = (datetime.datetime.now().strftime("%d%B_%I%M%p"))
    filepathdf=f'df_latlon/{l}_{h}_{now}_latlon_used.csv'
    odf.to_csv(filepathdf, index=False, header=odf.columns )   
    # Function also returns the dataframe
    return odf

In [4]:
# Call function for a managable subset of the dataset. Managable is situation dependent! 

# In hindsight I might have tried to run this for unique dealer_city entries only and then 
# did a one-many merge afterwards rather than this computationally expensive way. Fortunately 
# time wasn't an issue for me. 

# df_example = get_latlon_to_csv_used(0,10)

# GeoCoding to obtain Dealer County from latitude and longitude

In [9]:
# Load the accumulated car data
abspath = r'/Users/robertdavies/Desktop/DSI/GA_P2/LargeDatasets/00_all_cars_ll.csv'
df = pd.read_csv(abspath)

In [13]:
df.columns

Index(['name', 'name_subtitle', 'year', 'price', 'body', 'mileage', 'BHP',
       'doors', 'transmission', 'make', 'fuel', 'mpg', 'drivertrain', 'engine',
       'owners', 'ULEZ', 'county', 'dealer_area', 'dealer_city', 'dealer_lat',
       'dealer_lon', 'geocode', 'county.1', 'postcode', 'postcode_short',
       'postcode_shortest', 'seller1', 'used', 'e_engine_kWh', 'log_price',
       'log_mileage', 'orig_name', 'id', 'year_reg', 'link', 'href0'],
      dtype='object')

In [17]:
# Iterate through the latitude and longitude columns to extract unique occurences
dealer_latlon = []
for index, car in df.iterrows():
    latlon =  [car[-18],car[-17]]
    if latlon not in dealer_latlon:
        dealer_latlon.append(latlon)
len(dealer_latlon)

3270

In [18]:
len(df.dealer_lat.value_counts())

1720

In [19]:
dealer_latlon

[[52.688227, 52.688227],
 [52.3679896, 52.3679896],
 [53.92023225, 53.92023225],
 [52.5847949, 52.5847949],
 [50.9025349, 50.9025349],
 [52.2814519, 52.2814519],
 [51.4538022, 51.4538022],
 [51.2912247, 51.2912247],
 [51.3238969, 51.3238969],
 [52.628606, 52.628606],
 [56.4605938, 56.4605938],
 [52.9900566, 52.9900566],
 [53.6153659, 53.6153659],
 [51.0147895, 51.0147895],
 [53.749326, 53.749326],
 [51.6520851, 51.6520851],
 [51.4096275, 51.4096275],
 [51.87288, 51.87288],
 [51.2715316, 51.2715316],
 [51.1889529, 51.1889529],
 [53.78515425, 53.78515425],
 [52.9984193, 52.9984193],
 [52.9212617, 52.9212617],
 [53.30245885, 53.30245885],
 [55.5470714, 55.5470714],
 [51.58447, 51.58447],
 [53.3409714, 53.3409714],
 [56.6443013, 56.6443013],
 [51.5073219, 51.5073219],
 [53.4960455, 53.4960455],
 [53.3806626, 53.3806626],
 [50.45835615, 50.45835615],
 [51.3392845, 51.3392845],
 [52.5725769, 52.5725769],
 [51.52612575, 51.52612575],
 [52.22758, 52.22758],
 [54.5266327, 54.5266327],
 [51.5649

In [ ]:
abspath = r'/Users/robertdavies/Desktop/DSI/GA_P2/LargeDatasets/counties.csv'
counties = pd.read_csv(abspath)

In [ ]:
import reverse_geocoder as rg
import pprint
 
def reverseGeocode(coordinates):
    result = rg.search(coordinates)
     
    # result is a list containing ordered dictionary.
    pprint.pprint(result)
 
     
# Coordinates tuple.Can contain more than one pair.
coordinates =(53.0162014, -2.1812607)

reverseGeocode(coordinates)

In [27]:


gmaps = googlemaps.Client(key='#')

# Look up an address with reverse geocoding
reverse_geocode_result = gmaps.reverse_geocode((53.0162014, -2.1812607))



In [ ]:
#reverse_geocode_result

In [30]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = "53.0162014"
Longitude = "-2.1812607"

location = geolocator.reverse(Latitude+","+Longitude)
  
# Display
print(location.address)
print(location.raw['address']['postcode'])

Harcourt Street, Shelton, Hanley, Stoke-on-Trent, West Midlands, England, ST1 4NP, United Kingdom
ST1 4NP


In [ ]:
geocode = []
original_ll = []
for ll in tqdm(dealer_latlon):
    try:
        geocode.append(geolocator.reverse(str(ll[0])+","+str(ll[1])))
        original_ll.append(ll)
    except:
        geocode.append('bad result')
        original_ll.append(ll)
    


  2%|▉                                            | 64/3270 [00:31<26:15,  2.03it/s]

In [ ]:
geocode[-1]

In [ ]:
dealer_latlon2[-1]

In [ ]:
sub = geolocator.reverse('55.9533456,-3.1883749')
sub.address

In [ ]:
temp = pd.DataFrame({'base_ll':dealer_latlon,'original_ll':original_ll,'geocode':geocode})

In [ ]:
print(len(dealer_latlon))
print(len(original_ll))
print(len(geocode))

In [ ]:
temp.tail()

In [ ]:
temp = temp[temp.geocode!='bad result'].copy()

In [ ]:
temp['county']= 'bad result'
for index, loc in temp.iterrows():
    address = str(loc[-2])
    for county in counties.County:
        if county in address:
            temp.loc[index,'county'] = county
                     


In [ ]:
temp

In [ ]:
temp['postcode_short']= 'bad result'
for index, loc in temp.iterrows():
    address = str(loc[-3])
    try:
        po = re.findall('[A-Z][A-Z]?[0-9][0-9]?[ ]?[0-9][A-Z][A-Z]', address)[0]
        temp.loc[index,'po'] = po
    except:
        pass


In [ ]:
temp

In [ ]:
temp['postcode_short']= 'bad result'
for index, loc in temp.iterrows():
    address = str(loc[-2])
    try:
        po = address[:-3]
        temp.loc[index,'postcode_short'] = po
    except:
        pass


In [ ]:
temp

In [ ]:
temp.postcode_short.value_counts()

In [ ]:
tempbad = temp[temp.postcode_short=='bad res'].copy()

In [ ]:
tempbad

In [ ]:
from urllib.request import urlopen
import json
def getplace(lat, lon):
    url = "https://maps.googleapis.com/maps/api/geocode/json?key=#"
    url += "latlng=%s,%s&sensor=false" % (lat, lon)
    v = urlopen(url).read()
    j = json.loads(v)
    components = j['results'][0]['address_components']
    country = town = county =None
    #print(components)
    postcode = 'bad result'
    for c in components:
        if 'administrative_area_level_2' in c['types']:
            county = c['long_name']
        if "country" in c['types']:
            country = c['long_name']
        if "postal_code" in c['types']:
            postcode = c['long_name']
        if "postal_town" in c['types']:
            town = c['long_name']
            
    if postcode == 'bad result':
        components = j['results'][1]['address_components']
        for c in components:
            if 'administrative_area_level_2' in c['types']:
                county = c['long_name']
            if "country" in c['types']:
                country = c['long_name']
            if "postal_code" in c['types']:
                postcode = c['long_name']
    return postcode#, town, county, country

print(getplace(51.1, 0.1))

In [ ]:
tempbad['postcode2']='bad result'
for index, row in tempbad.iterrows():
    lat = row[1][0]
    lon = row[1][1]
    try:
        temp.loc[index,'po'] = getplace(lat, lon)
    except:
        temp.loc[index,'po'] = ' bad result'

In [ ]:
temp

In [ ]:
temp.po.value_counts()

In [ ]:
temp['postcode_short']= 'bad result'
for index, loc in temp.iterrows():
    address = str(loc[-2])
    try:
        po = address[:-3]
        temp.loc[index,'postcode_short'] = po
    except:
        pass

In [ ]:
temp.postcode_short.value_counts()

In [ ]:
temp

In [ ]:
tempbad2 = temp[temp.county=='bad result'].copy()

In [ ]:
from urllib.request import urlopen
import json
def getplace(lat, lon):
    url = "https://maps.googleapis.com/maps/api/geocode/json?key=#"
    url += "latlng=%s,%s&sensor=false" % (lat, lon)
    v = urlopen(url).read()
    j = json.loads(v)
    components = j['results'][0]['address_components']
    country = town = county =None
    #print(components)
    county = 'bad result'
    for c in components:
        if 'administrative_area_level_2' in c['types']:
            county = c['long_name']
        if "country" in c['types']:
            country = c['long_name']
        if "postal_code" in c['types']:
            postcode = c['long_name']
        if "postal_town" in c['types']:
            town = c['long_name']
            
    if county == 'bad result':
        components = j['results'][1]['address_components']
        for c in components:
            if 'administrative_area_level_2' in c['types']:
                county = c['long_name']
            if "country" in c['types']:
                country = c['long_name']
            if "postal_code" in c['types']:
                postcode = c['long_name']
    return county# postcode#, town, county, country

print(getplace(51.1, 0.1))

In [ ]:
for index, row in tempbad2.iterrows():
    lat = row[1][0]
    lon = row[1][1]
    try:
        temp.loc[index,'county'] = getplace(lat, lon)
    except:
        temp.loc[index,'county'] = ' bad result'

In [ ]:
temp

In [ ]:
temp.county.value_counts()

In [ ]:
temp.columns = ['base_ll', 'original_ll', 'geocode', 'county', 'postcode', 'postcode_short']

In [ ]:
merge_df = temp[['original_ll', 'geocode', 'county', 'postcode', 'postcode_short']].copy()
merge_df['dealer_lat']=merge_df.original_ll.apply(lambda x: x[0])
merge_df['dealer_lon']=merge_df.original_ll.apply(lambda x: x[1])
merge_df['dealer_lat_rnd']=merge_df.original_ll.apply(lambda x: round(x[0],5))
merge_df['dealer_lon_rnd']=merge_df.original_ll.apply(lambda x: round(x[1],5))

In [ ]:
merge_df.columns

In [ ]:
county_abspath = r'/Users/robertdavies/Desktop/DSI/GA_P2/LargeDatasets/04_merge_counties.csv'
merge_df.to_csv(county_abspath, index=False, header=merge_df.columns )

In [ ]:
county_abspath = r'/Users/robertdavies/Desktop/DSI/GA_P2/LargeDatasets/04_merge_counties.csv'
merge_df = pd.read_csv(county_abspath)

In [ ]:
merge_df

In [ ]:
# import googlemaps
# from datetime import datetime

# gmaps = googlemaps.Client(key='#')

# # Look up an address with reverse geocoding
# reverse_geocode_result = gmaps.reverse_geocode((53.0162014, -2.1812607))

In [ ]:
# for index, row in temp[temp.popo==''].iterrows():
#     address = str(row[-3])
#     for county in counties.County:
#         if county in address:
#             temp.loc[index,'popo'] = county